# Practica 1 Alfonso Ramos Rojas

## Carga de librerias

- **Numpy (np)**: operaciones numéricas y manipulación de arrays; aquí se usa para crear ventanas temporales (np.lib.stride_tricks.sliding_window_view) y cálculos vectorizados.
- **Pandas (pd)**: lectura y manejo de datos tabulares/series temporales (pd.read_csv, index datetime, selección de columnas).
- **Matplotlib.pyplot (plt)**: visualización de series temporales y marcado de anomalías (plot, scatter, show).
- **Tensorflow (tf)**: framework de deep learning y backend para Keras.
- **keras.models.Sequential**: contenedor secuencial para construir la RNN/modelo.
- **keras.layers.LSTM**: capa recurrente LSTM para modelado de dependencias temporales.
- **keras.layers.Dense**: capa densa/final para producir la predicción.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Input, RepeatVector, TimeDistributed
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest

ModuleNotFoundError: No module named 'numpy'

## 1. Contrucción de modelo LSTM y detección de anomalías

### a. Construcción del modelo básico

#### Cargar datos
Cargamos todo desde el csv de datos dado en el dataframe de pandas y escalamos los valores a un rango de [0,1], lo que mejorara la eficiencia de los modelos

In [ ]:
df = pd.read_csv("datos.csv",parse_dates=True)
print("Datos cargados: \n", df.head(5))

#### Creamos las ventanas temporales
Aquí el parametro principal es el tamaño de ventana, que varía mucho el resultado del modelo. En un principio probé con valores bajos (3, 5) y tras hacer pruebas, he descubierto que da mejores resultados tamaños medios de ventana de ventana. Primero con 24 daba mejores resultados, pero con 12 valores por ventana da los mejores resultados para nuestros datos.


In [ ]:
n_steps = 12
windows = np.lib.stride_tricks.sliding_window_view(df["value"], window_shape=(n_steps))
print("Dimensiones de ventanas: ", windows.shape)

#### Creamos datos de entrenamiento
Usamos como datos todas las ventanas menos la última para la predicción.
Para los valores objetivos usamos el primer valor siguiente a la ventana claro, que es el que puede predecir la primera ventana.
Además de eso redimensionamos las ventanas para que las acepte el modelo

In [ ]:
X = windows[:-1]
y = df["value"].values[n_steps:]
X = X.reshape((X.shape[0], X.shape[1], 1))
print("X Shape: ", X.shape)
print("Y Shape: ", y.shape)

#### Creamos el modelo
Creamos el modelo, con una capa de LSTM unida con una capa densa de 1 neurona al final como salida.
En primer lugar, las neuronas de la capa LSTM dan mucho mejor resultado siendo 100 en vez de 50

Además la funcion de activación sigmoide da unos resultados muy superiores a la tanh

In [ ]:
model = Sequential([
	LSTM(100, activation='sigmoid', input_shape=(n_steps, 1)),
	Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mse')

#### Entrenamos el modelo con las ventanas
Al principio usaba 50 epochs en el modelo, pensando que asi mejoraria mucho la calidad del modelo cuantos mas epochs use, sin embargo probando valores mas bajos, resulta que con tantisimos epochs se producia muchisimo sobre ajuste y además detectaba demasiados puntos como anomalías. Probando he encontrado que el numero óptimo de epochs es 15, manteniendo un mse medio final del modelo de aprox. 5

In [ ]:
epochs = 15

history = model.fit(X, y, epochs=epochs, verbose=1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

print("Modelo entrenado exitosamente")

#### Graficamos la perdida para optimizar el modelo
Este gráfico lo he estado usando para comprobar el numero optimo de iteraciones del modelo. Podemos observar que para el epoch 15 la pérdida no baja casi nada, por lo que no merece la pena seguir

In [ ]:
plt.plot(model.history.history['loss'], label='Pérdida de entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida (MSE)')
plt.title('Pérdida de Entrenamiento del Modelo LSTM')
plt.legend()
plt.show()

#### Creamos los valores de prediccion
Creamos las predicciones y calculamos el error de cada predicción con respecto al valor real

In [ ]:
predictions = model.predict(X)
errors = y - predictions.flatten()
print("Errores: \n", errors)

### b. Detección de anomalías

#### Threshold y deteccion de anomalias
Aquí usando el **rango intercuartílico** y usando el errror de la predicción usado antes, marcamos todos los puntos que esten 1.5 veces alejados de dicho rango como anomalías. Valores mayores de 1.5 provocaban demasiados falsos positivos.

In [ ]:
IRQ_DIFF = 1.5

error_IQR = np.percentile(errors, 75) - np.percentile(errors, 25)
print("Rango Intercuartil de los errores: ", error_IQR)
anomalies = np.where(np.abs(errors) > IRQ_DIFF * error_IQR)[0]
print("Número de anomalías detectadas: ", len(anomalies))
error_df = df.iloc[anomalies]

#### Graficamos anomalias
Graficamos los valores originales junto a las anomalias para observar el resultado final de nuestro modelo.

Observamos que la anomalía principal en el centro del grafico la detecta sin ninguna duda, y pequeños picos anómalos al final de la gráfica también. No detecta ciertos valores que no son demasiado anómalos como los del principio de la gráfica, pero el modelo tiene un buen desempeño para ver anomalías grandes.

In [ ]:
print(f"El numero de anomalias es {len(anomalies)} de {len(df)}")
print(df.iloc[anomalies])
plt.plot(df['timestamp'], df['value'], label='Valor real', color='blue')
plt.plot(predictions, linestyle='dashed', color='orange', label='Predicciones')
plt.scatter(
    df['timestamp'][anomalies],
    df['value'][anomalies],
    color='red',
    label='Anomalías')
plt.xticks(df['timestamp'][::len(df)//10], rotation=45)
plt.legend()
plt.show()

### c. Mejora del modelo
Los cambios que he aplicado para mejorar el modelo son los siguientes:

Principales:
- Reducir numero de epochs 50 -> 15
- Aumentar tamaño de ventana 3 -> 12

Adicionales:
- Neuronas LSTM 50 -> 100
- Funcion de activación 'tanh' -> 'sigmoid'

## 2. Autoencoder

#### Carga de datos
- Cargamos los datos como antes. 
- Creamos una division para tener datos de entrenamiento y de validacion (Entrenamiento 80% - Validación 20%).
- Escalar los valores a un rango de [0,1] para mejorar nuestro modelo, ya que mejoraba drasticamente el resultado final.

In [ ]:
df = pd.read_csv("datos.csv",parse_dates=['timestamp'], index_col='timestamp')
df = df.sort_index()

train_size = int(len(df) * 0.8)
train_df, test_df = df.iloc[:train_size].copy(), df.iloc[train_size:].copy()

min_train, max_train = train_df['value'].min(), train_df['value'].max()
range_train = max_train - min_train

scaler = MinMaxScaler(feature_range=(0, 1))

scaler.fit(train_df[['value']])

train_df['value_scaled'] = scaler.transform(train_df[['value']])
test_df['value_scaled'] = scaler.transform(test_df[['value']])

print(f"Datos de entrenamiento: {len(train_df)}, Datos de prueba: {len(test_df)}")

print("Datos Entrenamiento: \n", train_df.head(5))
print("Datos Prueba: \n", test_df.head(5))

#### Creacion de ventanas temporales
- Creamos ventanas diferentes para el entrenamiento y validacion.
- En este caso tras varias pruebas tambien me ha funcionado muy bien un tamaño de ventana de 12, consiguiendo una perdida de 0.2 mae aprox.

In [ ]:
n_steps = 12

train_data = train_df['value_scaled'].values
test_data = test_df['value_scaled'].values

X_train = np.lib.stride_tricks.sliding_window_view(train_data, window_shape=(n_steps)).copy()
X_test = np.lib.stride_tricks.sliding_window_view(test_data, window_shape=(n_steps)).copy()

print("Dimensiones X_train original: ", X_train.shape)
print("Dimensiones X_test original: ", X_test.shape)

X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

y_train = X_train
y_test = X_test


print("\n--- Dimensiones para el modelo ---")
print("X_train Shape: ", X_train.shape)
print("y_train Shape: ", y_train.shape)
print("X_test Shape: ", X_test.shape)
print("y_test Shape: ", y_test.shape)

#### Creamos el autoencoder
- He decidido usar un cuello de botella de 16, ya que en un principio puse 50 y descubir que habia que usar uno mucho mas pequeño para mi caso concreto
- Importante tambien el cambio al mae con respecto al ejercicio 1, este es mucho mejor para no ajustar el modelo a los outliers.

In [ ]:
BOTTLENECK_SIZE = 16

inputs = Input(shape=(n_steps, 1))
encoded = LSTM(BOTTLENECK_SIZE, activation='tanh')(inputs)
decoded = RepeatVector(n_steps)(encoded)
decoded = LSTM(BOTTLENECK_SIZE, activation='tanh', return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(1))(decoded)

autoencoder = Model(inputs=inputs, outputs=decoded)
autoencoder.compile(optimizer='adam', loss='mae')

autoencoder.summary()

#### Entrenamos el modelo
Ajustamos el modelo con los datos de prueba y validacion y en este caso si que merece la pena usar 50 epochs. La diferencia de tener un conjunto de datos de validación se nota muchisimo en el resultado final.

In [ ]:
epochs = 50
history = autoencoder.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=epochs, batch_size=32,
                        shuffle=False,
                        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)])

#### Predictions
- Creo las predicciones del modelo de datos y validacion, y creo nuevos dataframes para guardar los resultados de las predicciones, y si son anomalias o no.
- He cambiado el criterio de detección de nomalías a un percentil del 99%, en vez del rango intercuartílico, y tambien da buenos resultados

In [ ]:
predictions_train = autoencoder.predict(X_train)
predictions_test = autoencoder.predict(X_test)

mae_train = np.mean(np.abs(predictions_train - X_train), axis=(1,2))
mae_test = np.mean(np.abs(predictions_test - X_test), axis=(1,2))

print("\n--- Formas de MAE ---")
print("MAE Train: ", mae_train.shape)
print("MAE Test: ", mae_test.shape)

threshold = np.percentile(mae_train, 99)

print(f"Umbral de MAE para anomalías (99 percentil): {threshold:.4f}")

pred_last_step_train = predictions_train[:, -1, :]
pred_last_step_test = predictions_test[:, -1, :]

pred_original_scale_train = scaler.inverse_transform(pred_last_step_train)
pred_original_scale_test = scaler.inverse_transform(pred_last_step_test)

train_scores_df = pd.DataFrame(index=train_df.index[n_steps-1:])
train_scores_df['mae'] = mae_train
train_scores_df['threshold'] = threshold
train_scores_df['anomaly'] = train_scores_df['mae'] > train_scores_df['threshold']
train_scores_df['value'] = train_df['value'][n_steps - 1:]
train_scores_df['predicted'] = pred_original_scale_train

test_scores_df = pd.DataFrame(index=test_df.index[n_steps - 1:])
test_scores_df['mae'] = mae_test
test_scores_df['threshold'] = threshold
test_scores_df['anomaly'] = test_scores_df['mae'] > test_scores_df['threshold']
test_scores_df['value'] = test_df['value'][n_steps - 1:]
test_scores_df['predicted'] = pred_original_scale_test

all_scores_df = pd.concat([train_scores_df, test_scores_df])

print("\nAnomalías en datos de entrenamiento:", train_scores_df['anomaly'].sum())
print("\nAnomalías en datos de prueba:", test_scores_df['anomaly'].sum())

print("\n--- Anomalías detectadas en datos de prueba ---")
print(test_scores_df[test_scores_df['anomaly']].head(5))

#### Graficamos anomalias
Graficamos las anomalias de la misma manera que en el modelo anterior

In [ ]:
anomalies_df = all_scores_df[all_scores_df['anomaly']]
print("El numero de anomalias es ", len(anomalies_df), " de ", len(all_scores_df))
print(anomalies_df[['value']])

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8), sharex=True)

ax.set_title("Detección de Anomalías")
ax.plot(
    all_scores_df.index,
    all_scores_df['value'],
    label='Valor original',
    color='blue'
)
ax.plot(
    all_scores_df.index,
    all_scores_df['predicted'],
    label='Valor predicho',
    color='orange',
    linestyle='dashed'
)
ax.scatter(
    anomalies_df.index,
    anomalies_df['value'],
    label='Anomalía',
    color='red',
    marker='o',
    s=50
)
ax.legend()
ax.set_ylabel('Valor')
plt.xticks(rotation=45)
plt.show()

### Mejoras implementadas
- He usado en este caso un MinMaxScaler, para poder escalar los valores para que el autoencoder funcione mejor, y despues poder desescalarlos y ver la prediccion real de mi modelo.
- El tamaño de ventana lo he mantenido a 12, lo cual me ha dado muy buenos resultados en esta ocasión también
- He usado el mae en vez de el mse, ya que este ultimo penalizaba al modelo mucho por las anomalias y no hacia buenas predicciones.
- Y para el autoencoder he usado la funcion tanh predecia mejor, junto a un bottleneck de 16

## 3.Isolation Forest

### Cargamos y preparamos los datos
Aqui creo el dataframe como en los 2 ejercicios anteriores

In [ ]:
df = pd.read_csv("datos.csv",parse_dates=['timestamp'], index_col='timestamp')
df = df.sort_index()

print("Datos cargados y ordenados: \n", df.head(5))
print(f"Total de datos: {len(df)}")

### Creamos las ventanas y preparamos df de resultados
- Aqui creo las ventanas como siempre y creo el dataframe de resultados final
- El tamaño de ventana lo he mantenido en 12 ya que sigue funcionando muy bien.

In [ ]:
n_steps = 12

X_data = np.lib.stride_tricks.sliding_window_view(df["value"], window_shape=(n_steps))

print("Dimensiones de ventanas: ", X_data.shape)

df_results = pd.DataFrame({
	'value': df['value'][n_steps - 1:],
}, index=df.index[n_steps - 1:])

### Creamos modelo de IsolationForest
- Los estimadores no he encontrado casi diferencia al aumentarlos en este caso, siemplemente tarda mas el modelo.
- La contaminacion si que se nota mucho, con valores mas bajos como 0.05 o menos detectaba solo las anomalias mas exageradas. Con 0.1 detecta bastantes mas que son mas sutiles, y creo que en este contexto de un meidor de temperatura, no pasa nada por aumentar mas falsos positivos.

In [ ]:
n_estimators = 100
contamination = 0.1

iso_forest = IsolationForest(n_estimators=n_estimators, contamination=contamination, random_state=42)

### Entrenamos y predecimos los valores del modelo
- Entreno predigo y almaceno los valores que ha obtenido el IsolationForest, marcando si son anomalias o no

In [ ]:
predictions = iso_forest.fit_predict(X_data)
scores = iso_forest.decision_function(X_data)

iso_forest.save("isolation_forest_model.keras")

df_results['anomaly'] = predictions == -1
df_results['anomaly_score'] = scores

### Graficamos las anomalias detectadas
Graficamos las anomalias y el grafico original como siempre

In [ ]:
anomalies_df = df_results[df_results['anomaly']]
print(f"El numero de anomalias es {len(anomalies_df)} de {len(df_results)}")
print(anomalies_df[['value']])

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 8), sharex=True)

ax[0].set_title("Detección de Anomalías con Isolation Forest")
ax[0].plot(
    df_results.index, 
    df_results['value'], 
    label='Valor original', 
    color='blue'
)
ax[0].scatter(
    anomalies_df.index, 
    anomalies_df['value'], 
    label='Anomalía (detectada por IF)', 
    color='red', 
    marker='o',
    s=50
)
ax[0].legend()
ax[0].set_ylabel('Valor')

ax[1].set_title("Score de Anomalía de Isolation Forest")
ax[1].plot(
    df_results.index, 
    df_results['anomaly_score'], 
    label='Score de Anomalía', 
    color='green'
)
ax[1].scatter(
    anomalies_df.index, 
    anomalies_df['anomaly_score'],
    label='Anomalía',
    color='red',
    marker='o',
    s=50
)
ax[1].legend()
ax[1].set_ylabel('Score (más bajo = más anómalo)')
ax[1].set_xlabel('Timestamp')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()